## Importing the required Libraries

In [1]:
import pandas as pd
import numpy as np
import math as ma
from sklearn import datasets

## Loading iris dataset into DataFrame

In [2]:
iris=datasets.load_iris()   
#print(iris)
Data = pd.DataFrame(iris.data)
Data.columns = ['sl','sw','pl','pw']
print(Data)



      sl   sw   pl   pw
0    5.1  3.5  1.4  0.2
1    4.9  3.0  1.4  0.2
2    4.7  3.2  1.3  0.2
3    4.6  3.1  1.5  0.2
4    5.0  3.6  1.4  0.2
..   ...  ...  ...  ...
145  6.7  3.0  5.2  2.3
146  6.3  2.5  5.0  1.9
147  6.5  3.0  5.2  2.0
148  6.2  3.4  5.4  2.3
149  5.9  3.0  5.1  1.8

[150 rows x 4 columns]


In [3]:
y=pd.DataFrame(iris.target)
print(y)

     0
0    0
1    0
2    0
3    0
4    0
..  ..
145  2
146  2
147  2
148  2
149  2

[150 rows x 1 columns]


In [4]:
feature_list = ['sl','sw','pl','pw']

In [5]:
def count_Setosa(result):            
    #Counts number of setosa
    result=np.array(result[:])
    return (result==0).sum()    #returns the number of items whose value is 0(where 0 is label for setosa flower)

def count_Versicolor(result):       
    #Counts number of versicolor
    result=np.array(result[:])
    return (result==1).sum()    #returns the number of items whose value is 1(where 1 is label for versicolor flower)


# Entropy Function

In [6]:
Level = 0

In [7]:
def entropy(val):      #funtion to calculate entropy of given node
    entropy_=0
    if sum(val)==0:    #to avoid zerodivision error
        return 0
    for i in range(3):
        if val[i]/sum(val)==0:        #to avoid log(0) which is undefined
            continue
        entropy_+=((-1)*val[i]/sum(val))*ma.log(val[i]/sum(val),2)       
    return entropy_

# Gain ratio Function

In [8]:
def Gain_ratio(val,val1,val2):   # gain ratio funtion 
       
    info=entropy(val)       #info is entropy of root node
    info1=entropy(val1)     #info1 is entropy of first child node
    info2=entropy(val2)     # info1 is entropy of second child node
        
   
    a=(sum(val1)/(sum(val1)+sum(val2)))*info1     #info_gain=info-(a+b)  so we need to calculate a and b
                                                  # i.e weighted sum of individual nodes
    b=(sum(val2)/(sum(val1)+sum(val2)))*info2   
    
    info_gain=info-(a+b)
    
    if sum(val1)/(sum(val1)+sum(val2))==0:             
        split1=0 
    else:                                              # calculation splitinfo of first child node
        split1=((-1)*sum(val1)/(sum(val1)+sum(val2)))*ma.log(sum(val1)/(sum(val1)+sum(val2)),2) 
        
    if sum(val2)/(sum(val1)+sum(val2))==0:                                                      
        split2=0
    else:                                              # calculation splitinfo of second child node
        split2=((-1)*sum(val2)/(sum(val1)+sum(val2)))*ma.log(sum(val2)/(sum(val1)+sum(val2)),2)
    split_info=split1+split2
    try:
        gain_ratio_=info_gain/split_info                 
    except:
        gain_ratio_=0
  
    return gain_ratio_

# Gain Function

In [9]:
def Gain(Data,y,f):     
    
    data=Data[f]            #data is the coloumn data of feature f
   
    
    maxx=0                  #maxx will give max_gain ratio 
    minvalue=data.min()     #calculation of min and max value to run loop over all values of data
    m=minvalue+0.1
    maxvalue=data.max()
    feat=0         # later feat will return this feature f in df function
    mid=0          # at each time mid will store the value at which spliting is done by feature f
    
    
    while(m<=maxvalue):    
        val=[0,0,0]          #it will store number of 0`s , 1`s , 2`s of y on respective indexes 0,1,2
        val1=[0,0,0]         #it will store number of 0`s , 1`s , 2`s of  split_1y on respective indexes 0,1,2
        val2=[0,0,0]         #it will store number of 0`s , 1`s , 2`s of split_2y on respective indexes 0,1,2
        
        
        split_1x=Data[data<m]          # it is split of x data whose values are less then m
        split_1y=y[data<m]          # it is split of y data whose values are less then m
        
        
        
        
        split_2x=Data[data>=m]       # it is split of x data whose values are greater then m
        split_2y=y[data>=m]       # it is split of y data whose values are greater then m
        
        
        total_elements=len(Data)       # gives total number of elements in x
        val[0]=count_Setosa(y)          #countSetosa is function which returns number of setosa flowers defines at top
        val[1]=count_Versicolor(y) 
        val[2]=total_elements-val[0]-val[1]           #val[2] have value of 3rd type of flowers how many they are
        
        total_elements=len(split_1x)                  #same as for 1st split
        val1[0]=count_Setosa(split_1y)
        val1[1]=count_Versicolor(split_1y) 
        val1[2]=total_elements-val1[0]-val1[1]
        
        total_elements=len(split_2x)                  # same for 2nd split
        val2[0]=count_Setosa(split_2y)
        val2[1]=count_Versicolor(split_2y) 
        val2[2]=total_elements-val2[0]-val2[1]
       
        if val1.count(0)==3 and val2.count(0)==3:    #to prevent getting split_info to 0 in gain ratio
            continue
        max_gain=Gain_ratio(val,val1,val2)           #gain_ratio fun will give u max gain ratio using all 3 list which have all data
        if max_gain>=maxx:
            maxx=max_gain
            feat=f
            mid=m
        m+=0.1                                      #incrementing m by 0.1 so that at every stage we can get best gain_ratio
    return maxx,feat,mid

# Decision Tree Function

In [10]:
def decision_tree(Data,y,feature_list,Level):
    val=[0,0,0]                          #list contains the number of flowers of each type 
    no_of_features_left=len(feature_list)
    total_elements=len(Data)
    no_of_setosa=count_Setosa(y)          #countSetosa is function to count number of setosa flowers in output
    no_of_versicolor=count_Versicolor(y) 
    no_of_virginica=total_elements-no_of_setosa-no_of_versicolor
    val[0]=no_of_setosa
    val[1]=no_of_versicolor
    val[2]=no_of_virginica
  
    print('level ',Level)
    print('count of setosa =',no_of_setosa)
    print('count of versicolor =',no_of_versicolor)
    print('count of virginica =',no_of_virginica)
    print('current entropy is =',entropy(val))
    
    if val.count(0)==2:                 #if val has only one type of flowers it will reach leaf node
        print('Reached Leaf Node')
        return
    maxx_=0                   # maxx will store the maxx gain ratio
    mid=0                     #mid is the value at which feature splits ang gives max gain ratio
    feat=None
    
    for f in feature_list:
        max_gain,final_feature,m=Gain(Data,y,f) # gain fun to get max gainratio # max_gain is maximum gain ratio by final_feature    m is mid
        if maxx_<=max_gain:
            maxx_=max_gain
            feat=final_feature
            mid=m
        
  
    print('splitting on feature',feat,'with gain ratio',maxx_)         #feat is the feature at which split done
    
    new_1x=Data[Data[feat]<mid]     #spliting main data into two parts according to feat feature
   
    new_1y=y[Data[feat]<mid]     #spliting main output into two parts according to feat feature
   
    
    new_2x=Data[Data[feat]>=mid]
    
    new_2y=y[Data[feat]>=mid]
   
   
    features2=[Data for Data in feature_list]      # features will remain same bcz a feature can be used any number of times
    decision_tree(new_1x,new_1y,features2,Level+1)    #calling dt again recursively
    decision_tree(new_2x,new_2y,features2,Level+1)     #calling dt again recursively
    
    
  

    
decision_tree(Data,y,feature_list,0)     #main function call

level  0
count of setosa = 50
count of versicolor = 50
count of virginica = 50
current entropy is = 1.584962500721156
splitting on feature pw with gain ratio 0.9999999999999999
level  1
count of setosa = 50
count of versicolor = 0
count of virginica = 0
current entropy is = 0.0
Reached Leaf Node
level  1
count of setosa = 0
count of versicolor = 50
count of virginica = 50
current entropy is = 1.0
splitting on feature pw with gain ratio 0.6933647985912662
level  2
count of setosa = 0
count of versicolor = 49
count of virginica = 5
current entropy is = 0.44506485705083865
splitting on feature pl with gain ratio 0.6066178220203009
level  3
count of setosa = 0
count of versicolor = 49
count of virginica = 3
current entropy is = 0.31821529768323314
splitting on feature pl with gain ratio 0.2720453440631924
level  4
count of setosa = 0
count of versicolor = 47
count of virginica = 1
current entropy is = 0.14609425012013633
splitting on feature pw with gain ratio 1.0
level  5
count of setosa 